# Dataset Code Testbed

This notebook will conduct code prototyping for executing dataset related processes.

Lite version: `oliveira_lite.csv`

Whole version: `oliveira.csv`

*Note that the hyperparameter `random_state` was set to `1` instead of `None` for test repeatability.*

In [1]:
#Python Libraries
import time
import threading

#Data/Dataset libraries
import pandas as pd
import numpy as np

#Split Sampler/Data Splitting
from sklearn.model_selection import train_test_split

#Oversampler
from imblearn.over_sampling import RandomOverSampler, SMOTEN

#Label Encoding
from sklearn.preprocessing import LabelEncoder

#Filenames
filename = ["oliveira_lite.csv", "oliveira.csv"]
DATASET_FILENAME = filename[1] #Change accordingly;
API_LIST = "api_calls.txt"

#APIs List
API_FILE = open(API_LIST,"r")
APIS = API_FILE.readline().split(',')
APIS.append("NaN") #serves as a label for NaN values for Instance-based datasets
API_FILE.close()

#Timer
start = end = 0
def start_time():
    global start
    start = time.time()
def end_time():
    global start
    elapse = time.time()-start
    start = 0
    print(f"Elapsed Time: {round(elapse, 6)}s\n")
    return round(elapse, 6)

# Loading Dataset File

In [2]:
start_time()
oli = pd.read_csv(DATASET_FILENAME, low_memory=False, memory_map=True)
end_time()

#Dataset Information
print("Dataset Information:")
print(oli.info())
print("\nDataset Shape:", oli.shape)

#Dataset Labels:
print('\nDataset Labels:\n',oli['malware'].value_counts())

oli.head()

Elapsed Time: 0.053006s

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Columns: 102 entries, hash to malware
dtypes: int64(101), object(1)
memory usage: 2.0+ MB
None

Dataset Shape: (2610, 102)

Dataset Labels:
 malware
1    2540
0      70
Name: count, dtype: int64


,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
0,5b51d65972a349f90a86984c26b12b30,286,110,172,240,117,240,117,240,117,...,215,114,215,117,261,106,144,297,117,0
1,ceb8cc125478fad641daa4e04e9b2f19,198,208,106,271,144,194,257,127,114,...,215,86,215,172,117,215,86,215,297,0
2,f108600edf46d7c20f6acc522aeba6df,82,228,16,29,82,29,82,29,82,...,286,73,286,208,286,73,286,257,114,0
3,711be6337cb78a948f04759a0bd210ce,82,240,117,240,117,240,117,240,117,...,117,208,117,35,240,117,35,208,240,0
4,6de26f67ceb1e3303b889489010f4c3f,286,110,172,240,117,240,117,240,117,...,215,114,215,117,71,25,71,275,260,0


# Dataset Preparation

In [3]:
start_time()
#Dataset Cleaning: Nothing much here as the dataset is already clean enough for our purposes.
#Dataset Formatting: Changing order of columns to malware (label), hash, t_1, t_2, ..., t_99
label_col = oli.pop('malware')
oli.insert(0, label_col.name, label_col)
label_col = None
#Inverse Label Encoding
def inverse_label(item):
    global APIS
    return item.map(lambda x: APIS[int(x)])
oli.iloc[:, 2:] = oli.iloc[:, 2:].apply(inverse_label, axis=1, result_type='reduce')
print("")
end_time()
    
oli.head()


Elapsed Time: 0.613243s



,malware,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,0,5b51d65972a349f90a86984c26b12b30,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,CoCreateInstance,NtOpenSection,CreateDirectoryW,NtCreateFile,LdrGetProcedureAddress
1,0,ceb8cc125478fad641daa4e04e9b2f19,GetSystemInfo,NtAllocateVirtualMemory,NtOpenSection,GetTempPathW,CreateDirectoryW,GetFileAttributesW,FindFirstFileExW,DeleteFileW,...,LdrGetProcedureAddress,NtClose,NtCreateMutant,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,NtClose,NtCreateMutant,NtClose,NtCreateFile
2,0,f108600edf46d7c20f6acc522aeba6df,GetSystemTimeAsFileTime,NtProtectVirtualMemory,SetUnhandledExceptionFilter,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,...,GetFileAttributesExW,SetErrorMode,GetFileAttributesExW,SetErrorMode,NtAllocateVirtualMemory,SetErrorMode,GetFileAttributesExW,SetErrorMode,FindFirstFileExW,NtQueryDirectoryFile
3,0,711be6337cb78a948f04759a0bd210ce,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LdrLoadDll,LdrGetProcedureAddress,NtAllocateVirtualMemory,LdrGetProcedureAddress,GetSystemMetrics,LdrLoadDll,LdrGetProcedureAddress,GetSystemMetrics,NtAllocateVirtualMemory,LdrLoadDll
4,0,6de26f67ceb1e3303b889489010f4c3f,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW


# Feature Duplicate Processing

In [4]:
TB = oli.copy(deep=True) #Time-based behavior
IB = oli.copy(deep=True) #Instance-based behavior (to be created)

start_time()
print("Transposing IB...")
IB.transpose()
print("IB Transposed!")
print("Removing duplicates...")
print("Row:", end=" ")
for r in range(oli.shape[0]):
    #Per row (sample) removal of duplicates, thus cannot scale into the whole dataframe (which is way faster)
    row = IB.iloc[r, 2:].drop_duplicates(keep='first', inplace=False).to_list() 
    for i in range(100-len(row)):
        row.append("NaN")
    IB.iloc[r, 2:] = row
    print(r, end=" ")
print("\nDuplicates removed!")
print("Retransposing IB (revert)...")
IB.transpose()
print("IB Retransposed!")
end_time()

Transposing IB...
IB Transposed!
Removing duplicates...


Row: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 2

1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 

39.586391

In [5]:
TB.head()

,malware,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,0,5b51d65972a349f90a86984c26b12b30,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,CoCreateInstance,NtOpenSection,CreateDirectoryW,NtCreateFile,LdrGetProcedureAddress
1,0,ceb8cc125478fad641daa4e04e9b2f19,GetSystemInfo,NtAllocateVirtualMemory,NtOpenSection,GetTempPathW,CreateDirectoryW,GetFileAttributesW,FindFirstFileExW,DeleteFileW,...,LdrGetProcedureAddress,NtClose,NtCreateMutant,NtClose,LdrGetDllHandle,LdrGetProcedureAddress,NtClose,NtCreateMutant,NtClose,NtCreateFile
2,0,f108600edf46d7c20f6acc522aeba6df,GetSystemTimeAsFileTime,NtProtectVirtualMemory,SetUnhandledExceptionFilter,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,GetSystemTimeAsFileTime,GetTimeZoneInformation,...,GetFileAttributesExW,SetErrorMode,GetFileAttributesExW,SetErrorMode,NtAllocateVirtualMemory,SetErrorMode,GetFileAttributesExW,SetErrorMode,FindFirstFileExW,NtQueryDirectoryFile
3,0,711be6337cb78a948f04759a0bd210ce,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LdrLoadDll,LdrGetProcedureAddress,NtAllocateVirtualMemory,LdrGetProcedureAddress,GetSystemMetrics,LdrLoadDll,LdrGetProcedureAddress,GetSystemMetrics,NtAllocateVirtualMemory,LdrLoadDll
4,0,6de26f67ceb1e3303b889489010f4c3f,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW


In [6]:
IB.head()

,malware,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,0,5b51d65972a349f90a86984c26b12b30,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,NtOpenSection,NtMapViewOfSection,RegOpenKeyExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,ceb8cc125478fad641daa4e04e9b2f19,GetSystemInfo,NtAllocateVirtualMemory,NtOpenSection,GetTempPathW,CreateDirectoryW,GetFileAttributesW,FindFirstFileExW,DeleteFileW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,f108600edf46d7c20f6acc522aeba6df,GetSystemTimeAsFileTime,NtProtectVirtualMemory,SetUnhandledExceptionFilter,GetTimeZoneInformation,GetFileAttributesW,CreateDirectoryW,NtCreateFile,GetFileType,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,711be6337cb78a948f04759a0bd210ce,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,SetUnhandledExceptionFilter,NtAllocateVirtualMemory,NtQuerySystemInformation,LdrGetDllHandle,CoInitializeEx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,6de26f67ceb1e3303b889489010f4c3f,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,NtOpenSection,NtMapViewOfSection,RegOpenKeyExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Building Reserve Test as "External Dataset"

Using 90:10 split for Train:Reserve respectively

## Time-Based Dataset

In [7]:
start_time()

#Time-based
X = TB.iloc[:,1:] #Features (including hash for now)
y = TB.iloc[:,0] #Labels
TB_Train_X, TB_Reserve_X, TB_Train_y, TB_Reserve_y = train_test_split(X, y, test_size=.10, random_state=1, shuffle=True)

TB_Train_X.insert(0, 'malware', TB_Train_y)
TB_Reserve_X.insert(0, 'malware', TB_Reserve_y)

TB = TB_Train_X.copy(deep=True)
TB_Reserve = TB_Reserve_X.copy(deep=True)

TB_Reserve.drop(axis=1, labels='hash', inplace=True)
TB_Reserve.to_csv(index=False, chunksize=100, mode='w', path_or_buf='TB/TB_Reserve.csv')

end_time()

Elapsed Time: 1.627325s



1.627325

In [8]:
print("TB Shape:", TB.shape)
TB.head()

TB Shape: (2349, 102)


,malware,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
2530,1,bb31d354705eba393338ede3ffc1f908,LdrGetDllHandle,GetSystemInfo,NtDelayExecution,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,LdrGetDllHandle,...,NtClose,NtAllocateVirtualMemory,RegOpenKeyExW,RegCloseKey,RegQueryValueExW,RegCloseKey,RegOpenKeyExW,RegCloseKey,RegQueryValueExW,RegCloseKey
2346,1,9cfd5f4d96b84df576285452bea5ba2b,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,LdrGetProcedureAddress,...,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos
1260,1,03a70804f5247567d44633410482068f,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtAllocateVirtualMemory,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll
1875,1,84bde4396d6a67c7df0339fb614c4fc8,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
415,1,9e1c9b91dce6d7fed8c12e1e0f849bdb,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LdrGetProcedureAddress,GetSystemMetrics,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemDirectoryW,LdrLoadDll,CreateThread,FindResourceExW,NtClose,GetSystemMetrics


In [9]:
print("TB_Reserve Shape:", TB_Reserve.shape)
TB_Reserve.head()

TB_Reserve Shape: (261, 101)


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
1826,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
1196,1,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,LoadResource,FindResourceExW,LoadResource,FindResourceExW,LoadResource,FindResourceExW,LoadResource,FindResourceExW,LoadResource,FindResourceExW
1985,1,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,NtAllocateVirtualMemory,SetUnhandledExceptionFilter,...,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,LdrGetProcedureAddress,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,NtOpenProcess,NtClose,GetSystemMetrics
675,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
607,1,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegOpenKeyExW,RegQueryValueExW,RegCloseKey,NtCreateMutant,GetNativeSystemInfo,GetSystemWindowsDirectoryW,NtClose,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll


## Instance-Based Dataset

In [10]:
start_time()

#Instance-based
X = IB.iloc[:,1:] #Features (including hash for now)
y = IB.iloc[:,0] #Labels
IB_Train_X, IB_Reserve_X, IB_Train_y, IB_Reserve_y = train_test_split(X, y, test_size=.10, random_state=1, shuffle=True)

IB_Train_X.insert(0, 'malware', IB_Train_y)
IB_Reserve_X.insert(0, 'malware', IB_Reserve_y)

IB = IB_Train_X.copy(deep=True)
IB_Reserve = IB_Reserve_X.copy(deep=True)

IB_Reserve.drop(axis=1, labels='hash', inplace=True)
IB_Reserve.to_csv(index=False, chunksize=100, mode='w', path_or_buf='IB/IB_Reserve.csv')

end_time()

Elapsed Time: 0.044554s



0.044554

In [11]:
print("IB Shape:", IB.shape)
IB.head()

IB Shape: (2349, 102)


,malware,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
2530,1,bb31d354705eba393338ede3ffc1f908,LdrGetDllHandle,GetSystemInfo,NtDelayExecution,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,LdrGetProcedureAddress,CreateToolhelp32Snapshot,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2346,1,9cfd5f4d96b84df576285452bea5ba2b,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,SetUnhandledExceptionFilter,GetSystemDirectoryW,NtDelayExecution,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1260,1,03a70804f5247567d44633410482068f,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1875,1,84bde4396d6a67c7df0339fb614c4fc8,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,1,9e1c9b91dce6d7fed8c12e1e0f849bdb,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,SHGetFolderPathW,SetErrorMode,GetFileAttributesW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print("IB_Reserve Shape:", IB_Reserve.shape)
IB_Reserve.head()

IB_Reserve Shape: (261, 101)


,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
1826,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,1,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,1,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,SetUnhandledExceptionFilter,LdrLoadDll,RegOpenKeyExA,RegQueryValueExA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,1,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,NtOpenSection,NtMapViewOfSection,RegOpenKeyExW,RegQueryValueExW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Rebalancing

To be applied to the train split (i.e., TB & IB)

RandomOverSampler is less prone to oversampling a sample/set of samples as is the case shown in [Balance Oliveira](https://github.com/jm55DLSU/THESIS/blob/main/ML%20Test/Balancing%20Oliveira/Balancing_Oliveira.ipynb).

In [13]:
def sample_distribution(dataset):
    distribution = []
    total_size = dataset.shape[0]
    dataset = str(dataset[['hash']].value_counts()[0:10]).split('\n')
    dataset.pop(0)
    dataset.pop(len(dataset)-1)
    for d in dataset:
        hs = d.split(' ')[0]
        qt = int(d.split(' ')[len(d.split(' '))-1])
        ratio = str(round(qt/total_size*100,2))+"%"
        distribution.append([hs,qt,ratio])
        print([hs,qt,ratio])
def TB_rebalance(balancer):
    global TB
    #Time-based
    X = TB.iloc[:,1:]
    y = TB.iloc[:,0]
    print("TB Rebalance...")
    X, y = balancer.fit_resample(X, y)
    print("TB Rebalancing Finished!")
    X.insert(0, 'malware', y)
    TB = X.copy(deep=True)
    #Check sample distribution
    print("TB Sample Distribution (Top 10)")
    sample_distribution(TB)
    #Drop hash
    #TB.drop(axis=1, labels='hash', inplace=True)
    print("Time-based Label Counts:")
    print(TB['malware'].value_counts())
    TB.head()
    #Save to CSV
    TB.to_csv(index=False, chunksize=100, mode='w', path_or_buf='TB/TB.csv')
def IB_rebalance(balancer):
    global IB
    #Instance-based
    X = IB.iloc[:,1:]
    y = IB.iloc[:,0]
    print("IB Rebalance...")
    X, y = balancer.fit_resample(X, y)
    print("IB Rebalancing Finished!")
    X.insert(0, 'malware', y)
    IB = X.copy(deep=True)
    #Check sample distribution
    print("IB Sample Distribution (Top 10)")
    sample_distribution(IB)
    #Drop hash
    IB.drop(axis=1, labels='hash', inplace=True)
    print("Instance-based Label Counts:")
    print(IB['malware'].value_counts())
    IB.head()
    #Save to CSV
    IB.to_csv(index=False, chunksize=100, mode='w', path_or_buf='IB/IB.csv')
print("Time-based Label Counts:")
print(TB['malware'].value_counts())
print("Instance-based Label Counts:")
print(IB['malware'].value_counts())

Time-based Label Counts:
malware
1    2287
0      62
Name: count, dtype: int64
Instance-based Label Counts:
malware
1    2287
0      62
Name: count, dtype: int64


In [14]:
#ros = RandomOverSampler(sampling_strategy='minority', random_state=1, shrinkage=None)
#smoten = SMOTEN(sampling_strategy='minority', random_state=1, k_neighbors=5)

tb_thread = threading.Thread(target=TB_rebalance, args=(SMOTEN(sampling_strategy='minority', random_state=1, k_neighbors=2),))
ib_thread = threading.Thread(target=IB_rebalance, args=(SMOTEN(sampling_strategy='minority', random_state=1, k_neighbors=2),))

start_time()
tb_thread.start()
ib_thread.start()
tb_thread.join()
ib_thread.join()
end_time()

TB Rebalance...IB Rebalance...

TB Rebalancing Finished!


C:\Users\ejose\AppData\Local\Temp\ipykernel_5460\1177957425.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X.insert(0, 'malware', y)


TB Sample Distribution (Top 10)
['022fee1488e6a0d06bc5882b998663f3', 493, '10.78%']
['302566218f78bb35439df31b54685ad0', 266, '5.82%']
['2d2e4fd9a96a6638a6592ecf0a3bc846', 261, '5.71%']
['0d2ab02c993ea29a1989b442bf7150c7', 218, '4.77%']
['075323e77815ee8bcc7854ce23955a15', 168, '3.67%']
['0327301655f2e1c6bdbd4536a3349216', 144, '3.15%']
['5af545fe09a6680a1b4e1b3aaa2a8031', 117, '2.56%']
['0178875608d72fe44775793331f31718', 108, '2.36%']
['349aae8db20b24d14a90038d5c4c5549', 51, '1.11%']
['0b2ec965cee44e5bf3030bd1a61214f8', 48, '1.05%']
Time-based Label Counts:
malware
1    2287
0    2287
Name: count, dtype: int64
IB Rebalancing Finished!
IB Sample Distribution (Top 10)
['2d2e4fd9a96a6638a6592ecf0a3bc846', 361, '7.89%']
['0178875608d72fe44775793331f31718', 336, '7.35%']
['302566218f78bb35439df31b54685ad0', 266, '5.82%']
['608c58241918a7bf952ec094e0986ff8', 152, '3.32%']
['022fee1488e6a0d06bc5882b998663f3', 150, '3.28%']
['694c4c90716e11d84bb10b6ad03b308b', 112, '2.45%']
['6e51234733dec1e

C:\Users\ejose\AppData\Local\Temp\ipykernel_5460\1177957425.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X.insert(0, 'malware', y)


66.541942

In [15]:
TB

,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,LdrGetDllHandle,GetSystemInfo,NtDelayExecution,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,...,NtClose,NtAllocateVirtualMemory,RegOpenKeyExW,RegCloseKey,RegQueryValueExW,RegCloseKey,RegOpenKeyExW,RegCloseKey,RegQueryValueExW,RegCloseKey
1,1,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,LdrGetProcedureAddress,LdrGetDllHandle,...,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetCursorPos
2,1,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,...,NtAllocateVirtualMemory,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW,RegQueryValueExW,RegOpenKeyExW
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,LdrGetProcedureAddress,GetSystemMetrics,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemDirectoryW,LdrLoadDll,CreateThread,FindResourceExW,NtClose,GetSystemMetrics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,0,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW
4570,0,NtAllocateVirtualMemory,NtFreeVirtualMemory,NtAllocateVirtualMemory,GetFileType,NtAllocateVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemTimeAsFileTime,FindResourceExW,...,DrawTextExW,GetSystemMetrics,DrawTextExW,GetSystemMetrics,DrawTextExW,DrawTextExW,GetSystemMetrics,DrawTextExW,LdrGetProcedureAddress,DrawTextExW
4571,0,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW
4572,0,SetErrorMode,OleInitialize,LdrGetDllHandle,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,LdrLoadDll,LdrGetProcedureAddress,...,NtCreateFile,NtClose,NtQueryDirectoryFile,NtClose,LdrGetProcedureAddress,GetSystemWindowsDirectoryW,LoadStringW,GetSystemWindowsDirectoryW,GetSystemDirectoryW,RegOpenKeyExW


In [16]:
IB

,malware,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_90,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99
0,1,LdrGetDllHandle,GetSystemInfo,NtDelayExecution,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,LdrGetProcedureAddress,CreateToolhelp32Snapshot,Thread32First,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtFreeVirtualMemory,LdrGetDllHandle,LdrGetProcedureAddress,SetUnhandledExceptionFilter,GetSystemDirectoryW,NtDelayExecution,LdrLoadDll,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,LdrLoadDll,LdrGetProcedureAddress,NtProtectVirtualMemory,NtClose,NtOpenKey,NtQueryValueKey,LdrGetDllHandle,GetSystemInfo,NtAllocateVirtualMemory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,CryptAcquireContextW,NtOpenKey,NtQueryValueKey,NtClose,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,GetSystemTimeAsFileTime,LdrLoadDll,LdrGetProcedureAddress,LdrGetDllHandle,SetUnhandledExceptionFilter,SHGetFolderPathW,SetErrorMode,GetFileAttributesW,CreateDirectoryW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,0,GetSystemTimeAsFileTime,GetSystemInfo,LdrGetDllHandle,LdrGetProcedureAddress,NtAllocateVirtualMemory,GetFileAttributesW,FindFirstFileExW,GetSystemInfo,LdrGetProcedureAddress,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4570,0,NtAllocateVirtualMemory,NtFreeVirtualMemory,GetFileType,LdrGetDllHandle,LdrGetProcedureAddress,GetSystemTimeAsFileTime,FindResourceExW,LoadResource,NtCreateFile,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4571,0,GetSystemTimeAsFileTime,SetUnhandledExceptionFilter,LdrGetDllHandle,LdrGetProcedureAddress,FindResourceExW,FindResourceExW,LoadResource,NtFreeVirtualMemory,LdrLoadDll,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4572,0,CreateThread,IsDebuggerPresent,NtClose,GetSystemTimeAsFileTime,FindResourceExW,GetSystemTimeAsFileTime,NtAllocateVirtualMemory,NtCreateMutant,LdrLoadDll,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Label Encoding

For LightGBM's use

In [17]:
def label_encode(raw, enc_filename):
    global APIS
    encoded = [None, None, None, None]
    le = LabelEncoder()
    le.fit(APIS)
    for i, tb in enumerate(raw):
        print(f"Encoding: {enc_filename[i]}...")
        encoded[i] = tb.copy(deep=True)
        encoded[i].iloc[:,1:] = encoded[i].iloc[:,1:].apply(le.transform)
        print(f"\nWriting to file: {enc_filename[i]}...")
        encoded[i].to_csv(index=False, chunksize=100, mode='w', path_or_buf=enc_filename[i])
    return encoded
def TB_encode():
    global TB, TB_Reserve
    #Time-based
    print("Label Encoding Time-based Datasets...")
    tb_raw = [TB, TB_Reserve]
    tb_enc_filename = ['TB/TB_Enc.csv', 'TB/TB_Reserve_Enc.csv']
    tb_encoded = label_encode(tb_raw, tb_enc_filename)
    print("Label Encoding Time-based Datasets Finished!")
    print("")
def IB_encode():
    global IB, IB_Reserve
    #Instance-based
    print("Label Encoding Instance-based Datasets...")
    ib_raw = [IB, IB_Reserve]
    ib_enc_filename = ['IB/IB_Enc.csv', 'IB/IB_Reserve_Enc.csv']
    ib_encoded = label_encode(ib_raw, ib_enc_filename)
    print("Label Encoding Instance-based Datasets Finished!")
    print("")

In [18]:
tb_thread = threading.Thread(target=TB_encode)
ib_thread = threading.Thread(target=IB_encode)

start_time()
tb_thread.start()
ib_thread.start()
tb_thread.join()
ib_thread.join()
end_time()

Label Encoding Time-based Datasets...
Label Encoding Instance-based Datasets...
Encoding: TB/TB_Enc.csv...
Encoding: IB/IB_Enc.csv...

Writing to file: TB/TB_Enc.csv...

Writing to file: IB/IB_Enc.csv...
Encoding: TB/TB_Reserve_Enc.csv...

Writing to file: TB/TB_Reserve_Enc.csv...
Encoding: IB/IB_Reserve_Enc.csv...
Label Encoding Time-based Datasets Finished!
Writing to file: IB/IB_Reserve_Enc.csv...


Label Encoding Instance-based Datasets Finished!

Elapsed Time: 1.899878s



1.899878